# Data Preparation 

In [191]:
# load packages

import numpy as np
import pandas as pd
import calendar

In [193]:
# load dataset 

Chicago_Crimes_2005_to_2007 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2005_to_2007.csv", usecols = range(23))
Chicago_Crimes_2008_to_2011 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2008_to_2011.csv", usecols = range(23))
Chicago_Crimes_2012_to_2017 = pd.read_csv("crimes-in-chicago/Chicago_Crimes_2012_to_2017.csv", usecols = range(23))

# merge data 
Chicago_Crimes_2005_to_2017 = pd.concat([Chicago_Crimes_2005_to_2007, Chicago_Crimes_2008_to_2011, Chicago_Crimes_2012_to_2017])

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [195]:
# filter the dataset by years and export data from 2007 to 2017

Chicago_Crimes_2005_to_2017.head(20)
print(Chicago_Crimes_2005_to_2017.Year.unique())
Chicago_Crimes_2007_to_2017 = Chicago_Crimes_2005_to_2017.loc[(Chicago_Crimes_2005_to_2017['Year'] >= 2007) 
                                                              & (Chicago_Crimes_2005_to_2017['Year'] <= 2017)]
print(Chicago_Crimes_2007_to_2017.Year.unique())

# store the new DataFrame to a csv file for later use.
Chicago_Crimes_2007_to_2017.to_csv("Chicago_Crimes_2007_to_2017.csv",index=False)

[2006 2005 2007 2008 2009 2011 2010 2016 2015 2012 2014 2013 2017]
[2007 2008 2009 2011 2010 2016 2015 2012 2014 2013 2017]


In [196]:
# load data from 2007 to 2017
df = pd.read_csv("Chicago_Crimes_2007_to_2017.csv")

# check for features 
print(df.columns.values)

# drop unnecessary features, col 0
df.drop(df.columns[0], axis = 1, inplace = True)
print(df.dtypes)
print(df.shape) # (4767274, 22), 22 features 

# removing unneeded features
df.drop(['Block', 'X Coordinate', 'Y Coordinate', 'Updated On', 'Latitude', 'Longitude', 'Location'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['Unnamed: 0' 'ID' 'Case Number' 'Date' 'Block' 'IUCR' 'Primary Type'
 'Description' 'Location Description' 'Arrest' 'Domestic' 'Beat'
 'District' 'Ward' 'Community Area' 'FBI Code' 'X Coordinate'
 'Y Coordinate' 'Year' 'Updated On' 'Latitude' 'Longitude' 'Location']
ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude                object
Location                 object
dtype: object
(4767274, 22)


In [198]:
# check uniqueness of ID
df['ID'].is_unique #no unique, cases being recorded multiple times 

False

In [199]:
# return only one record per case by removing duplicated rows 
# ex: df[df['ID'] == 7781132]; 7815319, 7781132, 6023200  

print(len(df.ID.unique())) #3434853, but we have 4767274 rows 
df.drop_duplicates(keep = 'first', inplace = True) #remove duplicates 

# drop Case number, we are using ID as the primary key
df.drop(['Case Number'], axis = 1, inplace = True)
df.reset_index(inplace = True)

print(df.shape) #(3434853, 15) #matched 

3434853
(3434853, 15)


In [201]:
# check for missing values

df.isna().any() # Location Description, District, Ward, Community Area

index                   False
ID                      False
Date                    False
IUCR                    False
Primary Type            False
Description             False
Location Description     True
Arrest                  False
Domestic                False
Beat                    False
District                 True
Ward                     True
Community Area           True
FBI Code                False
Year                    False
dtype: bool

In [202]:
# Replace null values with 'Other' option; categorical 
df['Location Description'].value_counts() # 154 types + other
df['Location Description'].isna().sum() # 1933 null values 
df['Location Description'].fillna("Other", inplace = True)

# Numerical missing values 
df['District'].value_counts() # 25districts + nan
df['District'].isna().sum() # 43 nan

df['Ward'].value_counts() # 50 + nan
df['Ward'].isna().sum() # 56 nan

df['Community Area'].value_counts() # 78 + nan
df['Community Area'].isna().sum() # 926 nan

# drop columns Ward and Community Area
df.drop(['Ward', 'Community Area'], axis = 1, inplace = True)

# get all the missing data for District 
na = df[df['District'].isna()]
na = na[['Beat', 'District', 'Year']]

# fill in missing data based on the same beat# and year
for row in na.itertuples():
    district = df[(df['Beat'] == row.Beat) &
             (df['Year'] == row.Year)]['District'].unique()[0]
    df.iloc[row.Index, df.columns.get_loc('District')] = district


In [204]:
# separate Date column into two col: date and time 

df.Date.head(10)
datetime = pd.to_datetime(df.Date, errors='coerce')

df.drop(['Date'], axis = 1, inplace = True)
df['Date'] = datetime.dt.date
df['Time'] = datetime.dt.time

df['Month'] = datetime.dt.month
df['Day'] = datetime.dt.day
df['DayofWeek'] = datetime.dt.weekday_name

# convert month to abbr name 
df['Month'] = df['Month'].apply(lambda x: calendar.month_abbr[x])


In [205]:
df.drop(['index'], axis = 1, inplace = True)

In [206]:
iucr = pd.read_csv("IUCR.csv")
iucr['PRIMARY DESCRIPTION'].unique()

array(['HOMICIDE', 'CRIM SEXUAL ASSAULT', 'ROBBERY', 'BATTERY',
       'PUBLIC PEACE VIOLATION', 'RITUALISM', 'ASSAULT', 'STALKING',
       'BURGLARY', 'THEFT', 'MOTOR VEHICLE THEFT', 'ARSON',
       'HUMAN TRAFFICKING', 'DECEPTIVE PRACTICE', 'GAMBLING',
       'CRIMINAL DAMAGE', 'CRIMINAL TRESPASS', 'WEAPONS VIOLATION',
       'CONCEALED CARRY LICENSE VIOLATION', 'NON-CRIMINAL',
       'PROSTITUTION', 'OBSCENITY', 'PUBLIC INDECENCY',
       'OFFENSE INVOLVING CHILDREN', 'SEX OFFENSE', 'OTHER OFFENSE',
       'KIDNAPPING', 'NARCOTICS', 'OTHER NARCOTIC VIOLATION',
       'LIQUOR LAW VIOLATION', 'CRIMINAL ABORTION',
       'INTERFERENCE WITH PUBLIC OFFICER', 'INTIMIDATION',
       'OTHER OFFENSE '], dtype=object)

In [207]:
df['Primary Type'].unique()

array(['OTHER OFFENSE', 'CRIMINAL DAMAGE', 'NARCOTICS', 'BATTERY',
       'THEFT', 'CRIMINAL TRESPASS', 'BURGLARY', 'WEAPONS VIOLATION',
       'PROSTITUTION', 'DECEPTIVE PRACTICE', 'ASSAULT',
       'PUBLIC PEACE VIOLATION', 'ROBBERY', 'MOTOR VEHICLE THEFT',
       'ARSON', 'OFFENSE INVOLVING CHILDREN',
       'INTERFERENCE WITH PUBLIC OFFICER', 'CRIM SEXUAL ASSAULT',
       'SEX OFFENSE', 'KIDNAPPING', 'LIQUOR LAW VIOLATION', 'STALKING',
       'GAMBLING', 'OTHER NARCOTIC VIOLATION', 'INTIMIDATION',
       'OBSCENITY', 'PUBLIC INDECENCY', 'RITUALISM', 'HOMICIDE',
       'NON-CRIMINAL', 'HUMAN TRAFFICKING',
       'CONCEALED CARRY LICENSE VIOLATION', 'NON - CRIMINAL',
       'NON-CRIMINAL (SUBJECT SPECIFIED)'], dtype=object)

In [208]:
### since there are differences between the type names, resulting in extra categories for the same type 
### we want to make it the same for later use 

len(df['IUCR'].unique())#386


# merge two dataframes by IUCR
newDf = pd.merge(df, iucr, how = 'left', on = 'IUCR')

print(newDf.isna().any())
print(newDf['PRIMARY DESCRIPTION'].isna().sum())
print(newDf['SECONDARY DESCRIPTION'].isna().sum())
print(newDf['INDEX CODE'].isna().sum())

ID                       False
IUCR                     False
Primary Type             False
Description              False
Location Description     False
Arrest                   False
Domestic                 False
Beat                     False
District                 False
FBI Code                 False
Year                     False
Time                     False
Month                    False
Day                      False
DayofWeek                False
Date                     False
PRIMARY DESCRIPTION       True
SECONDARY DESCRIPTION     True
INDEX CODE                True
dtype: bool
2004311
2004311
2004311


In [210]:
### there are more iucr# in df than in iucr

# fill the new added cols with info from old cols 
newDf['PRIMARY DESCRIPTION'].fillna(df['Primary Type'], inplace=True)
newDf['SECONDARY DESCRIPTION'].fillna(df['Description'], inplace=True)

# replace nan in INDEX CODE with U, Unknow
newDf['INDEX CODE'].fillna('U', inplace = True)

# drop old columns 
newDf.drop(['Primary Type', 'Description'], axis = 1, inplace = True)

# rremove redundancy
newDf.loc[newDf['PRIMARY DESCRIPTION'] == 'NON-CRIMINAL (SUBJECT SPECIFIED)','PRIMARY DESCRIPTION'] = 'NON-CRIMINAL'
newDf.loc[newDf['PRIMARY DESCRIPTION'] == 'NON - CRIMINAL','PRIMARY DESCRIPTION'] = 'NON-CRIMINAL'
newDf.loc[newDf['PRIMARY DESCRIPTION'] == 'OTHER OFFENSE ','PRIMARY DESCRIPTION'] = 'OTHER OFFENSE'

In [211]:
# newDf.to_csv("cleanedData.csv",index=False)

In [212]:
newDf.shape

(3434853, 17)

In [215]:
newDf[newDf['Year'] == 2017]['Month'].value_counts()

Jan    11357
Name: Month, dtype: int64

In [217]:
# there are only records from Jan 2017, therefore remove data from 2017
newDf = newDf.loc[(newDf['Year'] >= 2007) & (newDf['Year'] < 2017)]

In [218]:
newDf.shape

(3423496, 17)

In [220]:
newDf.to_csv("cleanedData.csv",index=False)